<a href="https://colab.research.google.com/github/hrbolek/learning/blob/with_stack/notebooks/inf/04D_redux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
def addRow(state, row):
    return [*state, row]

def removeRow(state, row):
    id = row['id']
    return [item for item in state if item['id'] != id]

In [5]:
data = []
print(data)
data = addRow(data, {'id': 1, 'name': 'John'})
print(data)

[]
[{'id': 1, 'name': 'John'}]


In [6]:
data = removeRow(data, {'id': 1, 'name': 'John'})
print(data)

[]


## Stavová funkce

Stavová funkce je funkce, která na základě vstupu a aktuálního stavu generuje následující stav.

In [ ]:
def stateFunction(state, input):
    return [*state, input]

Stavová funkce výše přidává do stavové proměnné (listu) nový řádek.

Stavových funkcí pro danou datovou strukturu je možné mít více.

In [22]:
def addRow(state, input):
    return [*state, input]

def removeRow(state, row):
    id = row['id']
    return [item for item in state if item['id'] != id]

def updateRow(state, row):
    id = row['id']
    return [{**item, **row} if item['id'] == id else item for item in state]

Takto definované funkce jsou funkce, jejichž chování nezáleží na stavu žádné proměnné definované mimo funkci.

Významnou výhodou je, že tyto funkce lze snadno testovat.

Test stavových funkcí

In [23]:
data = []
print(data)
data = addRow(data, {'id': 1, 'name': 'John'})
print(data)
data = updateRow(data, {'id': 1, 'name': 'Julia'})
print(data)
data = removeRow(data, {'id': 1})
print(data)

[]
[{'id': 1, 'name': 'John'}]
[{'id': 1, 'name': 'Julia'}]
[]


## Příklad
Mějme simulaci komponenty, která pro přístup k datům poskytuje dvě funkce. Funkci pro čtení dat a funkci pro změnu dat.

Funkce pro čtení dat je bezparametrická.

Funkce pro změnu dat očekává jako parametr stavovou jednoparametrickou funkci. To je v rozporu se skutečností, že stavová funkce je dvouparametrická.

In [7]:
def createAccess(initialData):
    data = {'data': initialData}
    def dispatch(reducer):
        data['data'] = reducer(data['data'])
    def getData():
        return data['data']
    return (dispatch, getData)

Test stavových funkcí

In [8]:
(dispatch, getData) = createAccess([])
print(getData())
dispatch(lambda state: addRow(state, {'id': 1, 'name': 'John'}))
print(getData())
dispatch(lambda state: addRow(state, {'id': 2, 'name': 'Julia'}))
print(getData())
dispatch(lambda state: removeRow(state, {'id': 1}))
print(getData())


[]
[{'id': 1, 'name': 'John'}]
[]


In [19]:
def execute(dispatch, reducer=None, action=None):
    def justReducer(reducer):
        return execute(dispatch, reducer, action)
    def justAction(action):
        return execute(dispatch, reducer, action)

    def twoParams(reducer=None, action=None):
        def justReducer(reducer):
            return execute(dispatch, reducer, action)
        def justAction(action):
            return execute(dispatch, reducer, action)
        if reducer is None:
            return justReducer
        else:
           return justAction

    if ((reducer is None) & (action is None)):
        return twoParams
    elif (reducer is None):
        return justReducer
    elif (action is None):
        return justAction

    if callable(reducer):
        return dispatch(lambda state: reducer(state, action))
    else:
        # reducer and action are swapped
        return dispatch(lambda state: action(state, reducer))


In [21]:
(dispatch, getData) = createAccess([])
print(getData())
execute(dispatch)(addRow)({'id': 1, 'name': 'John'})
print(getData())
execute(dispatch, addRow)({'id': 2, 'name': 'John'})
print(getData())
execute(dispatch, addRow, {'id': 3, 'name': 'John'})
print(getData())

[]
[{'id': 1, 'name': 'John'}]
[{'id': 1, 'name': 'John'}, {'id': 2, 'name': 'John'}]
[{'id': 1, 'name': 'John'}, {'id': 2, 'name': 'John'}, {'id': 3, 'name': 'John'}]
